**Please visit https://ProStepSky.github.io/.**

This work © 2024 by "ProStepSky.github.io จัดทำโดย S.U." is licensed under [CC BY-NC-ND 4.0](http://creativecommons.org/licenses/by-nc-nd/4.0/)  
(อ้างอิงแหล่งที่มา ห้ามนำไปใช้เพื่อการค้า และห้ามดัดแปลง)

---

# ➡ <font color='dodgerblue'><u>Expected Loss</u></font> ⬅

$EL = PD \times LGD \times EAD$

- `EL`: Expected Loss (ค่าความเสียหายที่คาดว่าจะเกิดขึ้น)
- `PD`: Probability of Default (ความน่าจะเป็นที่ลูกหนี้จะผิดนัดชำระหนี้)
- `LGD`: Loss Given Default (เปอร์เซนต์ความเสียหายเมื่อลูกหนี้ผิดนัดชำระหนี้)
- `EAD`: Exposure at Default (ยอดหนี้เมื่อลูกหนี้ผิดชำระหนี้)

---

# Import Libraries

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Suppress all warnings 
import warnings
warnings.filterwarnings("ignore")

# Import Data

 นำเข้า dataset ที่ผ่านการ preprocessed จาก notebook ชื่อ 01 - PD - Data Preparation - Train Dataset แล้ว:

In [3]:
loan_data_preprocessed_backup = pd.read_csv('./data/loan_data_2007_2014_preprocessed.csv')

In [4]:
loan_data_preprocessed = loan_data_preprocessed_backup.copy()

In [5]:
loan_data_preprocessed.head()

,Unnamed: 0.1,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,...,addr_state:UT,addr_state:VA,addr_state:VT,addr_state:WA,addr_state:WI,addr_state:WV,addr_state:WY,initial_list_status:f,initial_list_status:w,good_bad
0,0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,...,False,False,False,False,False,False,False,True,False,1
1,1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,...,False,False,False,False,False,False,False,True,False,0
2,2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,...,False,False,False,False,False,False,False,True,False,1
3,3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,...,False,False,False,False,False,False,False,True,False,1
4,4,4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,...,False,False,False,False,False,False,False,True,False,1


สำหรับตัวแปร `mths_since_last_delinq` (จำนวนเดือนที่ผ่านไปนับจากการขาดการชำระเงินล่าสุด) และ `mths_since_last_record` (จำนวนเดือนที่ผ่านไปนับจากที่มีการบันทึกประวัติเสียทางการเงินล่าสุด) ในการสร้าง LGD และ EAD models นี้ จะนำมาใช้ในแบบ continuous values เลย

สำหรับ missing values ของสองตัวแปรนั้น จะต้องตัดสินใจจากเหตุการณ์แวดล้อมด้วยว่า ควรแทนค่า missing values อย่างไร

แต่ ณ ที่นี้ จะแทนค่า missing values ด้วย 0 ตามทัศนคติ conservative risk management ที่ว่าควรสมมติให้สิ่งที่ยังไม่ทราบข้อมูลให้มีความเสี่ยงสูงไว้ก่อน:

In [6]:
loan_data_preprocessed['mths_since_last_delinq'].fillna(0, inplace = True)

In [7]:
loan_data_preprocessed['mths_since_last_record'].fillna(0, inplace = True)

สร้างตัวแปร features_all เพื่อเก็บเฉพาะชื่อของตัวแปรอิสระที่ต้องใช้เท่านั้น:

In [8]:
features_all = ['grade:A',
'grade:B',
'grade:C',
'grade:D',
'grade:E',
'grade:F',
'grade:G',
'home_ownership:MORTGAGE',
'home_ownership:NONE',
'home_ownership:OTHER',
'home_ownership:OWN',
'home_ownership:RENT',
'verification_status:Not Verified',
'verification_status:Source Verified',
'verification_status:Verified',
'purpose:car',
'purpose:credit_card',
'purpose:debt_consolidation',
'purpose:educational',
'purpose:home_improvement',
'purpose:house',
'purpose:major_purchase',
'purpose:medical',
'purpose:moving',
'purpose:other',
'purpose:renewable_energy',
'purpose:small_business',
'purpose:vacation',
'purpose:wedding',
'initial_list_status:f',
'initial_list_status:w',
'term_int',
'emp_length_int',
'mths_since_issue_d',
'mths_since_earliest_cr_line',
'funded_amnt',
'int_rate',
'installment',
'annual_inc',
'dti',
'delinq_2yrs',
'inq_last_6mths',
'mths_since_last_delinq',
'mths_since_last_record',
'open_acc',
'pub_rec',
'total_acc',
'acc_now_delinq',
'total_rev_hi_lim']

สร้างตัวแปร features_reference_cat เพื่อเก็บชื่อของ dummy variable reference categories:

In [9]:
features_reference_cat = ['grade:G',
'home_ownership:RENT',
'verification_status:Verified',
'purpose:credit_card',
'initial_list_status:f']

เลือกเก็บเฉพาะตัวแปรที่ต้องใช้เท่านั้น นั่นคือตัวแปรที่มีชื่ออยู่ใน features_all list:

In [10]:
loan_data_preprocessed_lgd_ead = loan_data_preprocessed[features_all]

ต้องทำการกำจัด 1 dummy variable สำหรับตัวแปรต้นฉบับแต่ละตัว เพื่อป้องกันปัญหา dummy variable trap ซึ่งทำให้เกิด multicollinearity

ทำการ drop ตัวแปรที่เป็น dummy variable reference categories:

In [11]:
loan_data_preprocessed_lgd_ead = loan_data_preprocessed_lgd_ead.drop(features_reference_cat, axis = 1)

---

ดัดแปลง .fit() method จาก Logistic Regression class เพื่อให้คำนวณ p-values ได้:

In [12]:
# P values for sklearn logistic regression.

# Class to display p-values for logistic regression in sklearn.

from sklearn import linear_model
import scipy.stats as stat

class LogisticRegression_with_p_values:
    
    # Inherit everything from the original LogisticRegression class.
    def __init__(self,*args,**kwargs):#,**kwargs):
        
        # -------------------------------------------------------
        # This function will be provided upon request. Thank you.
        # -------------------------------------------------------

    # Overwrite .fit() with a function that calculate p-values.
    def fit(self,X,y):
        
        # -------------------------------------------------------
        # This function will be provided upon request. Thank you.
        # -------------------------------------------------------

ดัดแปลง .fit() method จาก Linear Regression class เพื่อให้คำนวณ p-values ได้:

In [13]:
# Since the p-values are obtained through certain statistics, we need the 'stat' module from scipy.stats
import scipy.stats as stat

# Since we are using an object oriented language such as Python, we can simply define our own 
# LinearRegression class (the same one from sklearn)
# By typing the code below we will ovewrite a part of the class with one that includes p-values
# Here's the full source code of the ORIGINAL class: https://github.com/scikit-learn/scikit-learn/blob/7b136e9/sklearn/linear_model/base.py#L362


class LinearRegression(linear_model.LinearRegression):
    """
    LinearRegression class after sklearn's, but calculate t-statistics
    and p-values for model coefficients (betas).
    Additional attributes available after .fit()
    are `t` and `p` which are of the shape (y.shape[1], X.shape[1])
    which is (n_features, n_coefs)
    This class sets the intercept to 0 by default, since usually we include it
    in X.
    """
    
    # nothing changes in __init__
    def __init__(self, fit_intercept=True, normalize=False, copy_X=True,
                 n_jobs=1):
        
        # -------------------------------------------------------
        # This function will be provided upon request. Thank you.
        # -------------------------------------------------------

    
    def fit(self, X, y, n_jobs=1):
        
        # -------------------------------------------------------
        # This function will be provided upon request. Thank you.
        # -------------------------------------------------------
        
        return self

เปิด LGD models ทั้ง stage 1 และ 2, และ EAD model จาก disk:

In [14]:
import pickle

In [15]:
filehandler = open('./data/lgd_model_stage_1.pkl','rb')
reg_lgd_st_1 = pickle.load(filehandler)
filehandler.close()

In [16]:
filehandler = open('./data/lgd_model_stage_2.pkl','rb')
reg_lgd_st_2 = pickle.load(filehandler)
filehandler.close()

In [17]:
filehandler = open('./data/ead_model.pkl','rb')
reg_ead = pickle.load(filehandler)
filehandler.close()

---

## LGD (Loss Given Default)

ทำการทำนายโดยใช้ LGD stage 1 model (Logistic Regression):

In [18]:
loan_data_preprocessed['recovery_rate_st_1'] = reg_lgd_st_1.model.predict(loan_data_preprocessed_lgd_ead)

ทำการทำนายโดยใช้ LGD stage 2 model (Linear Regression):

In [19]:
loan_data_preprocessed['recovery_rate_st_2'] = reg_lgd_st_2.predict(loan_data_preprocessed_lgd_ead)

ทำการรวมผลการทำนาย (โดยการคูณ) ทั้ง LGD stage 1 และ stage 2 เข้าด้วยกัน จะได้คำตอบสุดท้ายของผลทำนายค่า recovery rate:

In [20]:
loan_data_preprocessed['recovery_rate'] = loan_data_preprocessed['recovery_rate_st_1'] * loan_data_preprocessed['recovery_rate_st_2']

เนื่องจาก linear regression model ไม่ได้ถูกจำกัดไว้ว่าต้องให้ output อยู่ระหว่าง 0 ถึง 1 เท่านั้น แม้ว่าข้อมูลที่นำมา train จะอยู่ระหว่าง 0 ถึง 1 ก็ตาม

ดังนั้น จึงต้องตัดค่าที่อยู่นอกพิสัย 0 ถึง 1 ให้เท่ากับ 0 หรือ 1 (นั่นคือ มากกว่า 1 ให้เปลี่ยนเป็น 1 และ น้อยกว่า 0 ให้เปลี่ยนเป็น 0):

In [21]:
loan_data_preprocessed['recovery_rate'] = np.where(loan_data_preprocessed['recovery_rate'] < 0, 0, loan_data_preprocessed['recovery_rate'])
loan_data_preprocessed['recovery_rate'] = np.where(loan_data_preprocessed['recovery_rate'] > 1, 1, loan_data_preprocessed['recovery_rate'])

Loss Given Default (LGD) = 1 - Recovery Rate:

In [22]:
loan_data_preprocessed['LGD'] = 1 - loan_data_preprocessed['recovery_rate']

แสดงสถิติของผลทำนายค่า Loss Given Default (LGD: เปอร์เซนต์ความเสียหายเมื่อลูกหนี้ผิดนัดชำระหนี้):

In [23]:
loan_data_preprocessed['LGD'].describe()

count    466285.000000
mean          0.933493
std           0.058097
min           0.362427
25%           0.881504
50%           0.915927
75%           1.000000
max           1.000000
Name: LGD, dtype: float64

จากสถิติข้างบน ตอนนี้ผลทำนายค่า Loss Given Default อยู่ระหว่าง 0 ถึง 1 แล้ว

---

## EAD (Exposure at Default)

ค่า CCF (Credit Conversion Factor: ค่าแปลงสภาพ) คืออัตราส่วนของเงินต้นที่ยังค้างชำระต่อจำนวนเงินที่ปล่อยกู้ ณ ขณะที่ผู้ขอกู้ผิดนัดชำระหนี้ถ้าหากว่าผู้ขอกู้ผิดนัดชำระหนี้; นั่นคือ หากค่า CCF ยิ่งสูง ยอดหนี้ ณ ขณะที่ผู้ขอกู้ผิดนัดชำระหนี้ยิ่งสูง; นั่นคือ ยิ่งเสี่ยง

ทำการทำนายค่า Credit Conversion Factor (CCF: ค่าแปลงสภาพ):

In [24]:
loan_data_preprocessed['CCF'] = reg_ead.predict(loan_data_preprocessed_lgd_ead)

เนื่องจาก linear regression model ไม่ได้ถูกจำกัดไว้ว่าต้องให้ output อยู่ระหว่าง 0 ถึง 1 เท่านั้น แม้ว่าข้อมูลที่นำมา train จะอยู่ระหว่าง 0 ถึง 1 ก็ตาม

ดังนั้น จึงต้องตัดค่าที่อยู่นอกพิสัย 0 ถึง 1 ให้เท่ากับ 0 หรือ 1 (นั่นคือ มากกว่า 1 ให้เปลี่ยนเป็น 1 และ น้อยกว่า 0 ให้เปลี่ยนเป็น 0):

In [25]:
loan_data_preprocessed['CCF'] = np.where(loan_data_preprocessed['CCF'] < 0, 0, loan_data_preprocessed['CCF'])
loan_data_preprocessed['CCF'] = np.where(loan_data_preprocessed['CCF'] > 1, 1, loan_data_preprocessed['CCF'])

Exposure at Default (EAD) = Credit Conversion Factor * total funded amount:

In [26]:
loan_data_preprocessed['EAD'] = loan_data_preprocessed['CCF'] * loan_data_preprocessed_lgd_ead['funded_amnt']

แสดงสถิติของผลทำนายค่า Exposure at Default (EAD):

In [27]:
loan_data_preprocessed['EAD'].describe()

count    466285.000000
mean      10818.007603
std        6937.951334
min         190.258506
25%        5496.760863
50%        9210.556594
75%       14697.371881
max       35000.000000
Name: EAD, dtype: float64

In [28]:
loan_data_preprocessed.head()

,Unnamed: 0.1,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,...,addr_state:WY,initial_list_status:f,initial_list_status:w,good_bad,recovery_rate_st_1,recovery_rate_st_2,recovery_rate,LGD,CCF,EAD
0,0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,...,False,True,False,1,1,0.085883,0.085883,0.914117,0.588772,2943.860059
1,1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,...,False,True,False,0,1,0.084099,0.084099,0.915901,0.776507,1941.268368
2,2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,...,False,True,False,1,1,0.080193,0.080193,0.919807,0.657455,1577.892265
3,3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,...,False,True,False,1,1,0.094563,0.094563,0.905437,0.659145,6591.452793
4,4,4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,...,False,True,False,1,1,0.088069,0.088069,0.911931,0.706868,2120.604693


---

## PD (Probability of Default)

### Import Data

นำเข้า datasets ที่ได้ผ่านการ train/test split และ สร้าง dummy variables ใน notebooks ชื่อ "01 - PD - Data Preparation - Train Dataset" และ "02 - PD - Data Preparation - Test Dataset" แล้ว:

In [29]:
loan_data_inputs_train = pd.read_csv('./data/loan_data_inputs_train.csv')

In [30]:
loan_data_inputs_test = pd.read_csv('./data/loan_data_inputs_test.csv')

ทำการรวม train และ test datasets เป็น DataFrame เดียวกันเพื่อใช้เป็น input ต่อไป:

In [31]:
# Concatenate two DataFrames along the rows (axis = 0).
loan_data_inputs_pd = pd.concat([loan_data_inputs_train, loan_data_inputs_test], axis = 0)

In [32]:
loan_data_inputs_pd.shape

(466285, 323)

จากข้างบนพบว่า มิติของ DataFrame มีจำนวน rows เท่ากับ train และ test datasets ก่อนที่จะ concatenate

In [33]:
loan_data_inputs_pd.head()

,Unnamed: 0.1,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,...,mths_since_last_delinq:4-30,mths_since_last_delinq:31-56,mths_since_last_delinq:>=57,mths_since_last_record:Missing,mths_since_last_record:0-2,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86
0,427211,427211,12796369,14818505,24000,24000,24000.0,36 months,8.90,762.08,...,0,0,0,1,0,0,0,0,0,0
1,206088,206088,1439740,1691948,10000,10000,10000.0,36 months,14.33,343.39,...,1,0,0,1,0,0,0,0,0,0
2,136020,136020,5214749,6556909,20425,20425,20425.0,36 months,8.90,648.56,...,0,1,0,1,0,0,0,0,0,0
3,412305,412305,13827698,15890016,17200,17200,17200.0,36 months,16.59,609.73,...,1,0,0,1,0,0,0,0,0,0
4,36159,36159,422455,496525,8400,8400,7450.0,36 months,12.84,282.40,...,0,0,0,1,0,0,0,0,0,0


จากข้างบน Train และ test datasets ที่นำเข้ามา ได้ผ่านการ shuffling และ splitting ตอนทำ train/test split จึงทำให้มีลำดับ indices ที่ shuffled ตาม "Unnamed: 0"

เพื่อไม่ให้สูญเสีย index information จึงตั้ง index ให้เท่ากับค่าใน column ชื่อ "Unnamed: 0" ดังนี้:

In [34]:
loan_data_inputs_pd = loan_data_inputs_pd.set_index('Unnamed: 0')

In [35]:
loan_data_inputs_pd.head()

,Unnamed: 0.1,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,mths_since_last_delinq:4-30,mths_since_last_delinq:31-56,mths_since_last_delinq:>=57,mths_since_last_record:Missing,mths_since_last_record:0-2,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
427211,427211,12796369,14818505,24000,24000,24000.0,36 months,8.90,762.08,A,...,0,0,0,1,0,0,0,0,0,0
206088,206088,1439740,1691948,10000,10000,10000.0,36 months,14.33,343.39,C,...,1,0,0,1,0,0,0,0,0,0
136020,136020,5214749,6556909,20425,20425,20425.0,36 months,8.90,648.56,A,...,0,1,0,1,0,0,0,0,0,0
412305,412305,13827698,15890016,17200,17200,17200.0,36 months,16.59,609.73,D,...,1,0,0,1,0,0,0,0,0,0
36159,36159,422455,496525,8400,8400,7450.0,36 months,12.84,282.40,C,...,0,0,0,1,0,0,0,0,0,0


Dummy variables รวมทั้ง reference category จาก PD model ที่ได้คัดเลือกแล้วว่ามีส่วนช่วยในการทำนายการผิดนัดชำระหนี้:

In [36]:
features_all_pd = ['grade:A',
'grade:B',
'grade:C',
'grade:D',
'grade:E',
'grade:F',
'grade:G',
'home_ownership:RENT_OTHER_NONE_ANY',
'home_ownership:OWN',
'home_ownership:MORTGAGE',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'addr_state:NM_VA',
'addr_state:NY',
'addr_state:OK_TN_MO_LA_MD_NC',
'addr_state:CA',
'addr_state:UT_KY_AZ_NJ',
'addr_state:AR_MI_PA_OH_MN',
'addr_state:RI_MA_DE_SD_IN',
'addr_state:GA_WA_OR',
'addr_state:WI_MT',
'addr_state:TX',
'addr_state:IL_CT',
'addr_state:KS_SC_CO_VT_AK_MS',
'addr_state:WV_NH_WY_DC_ME_ID',
'verification_status:Not Verified',
'verification_status:Source Verified',
'verification_status:Verified',
'purpose:smb_edu_mov_house_renew_wedd',
'purpose:credit_card',
'purpose:debt_consolidation',
'purpose:oth_med_vacation',
'purpose:homeimpv_majorpurch_car',
'initial_list_status:f',
'initial_list_status:w',
'term:36',
'term:60',
'emp_length:0',
'emp_length:1',
'emp_length:2-4',
'emp_length:5-6',
'emp_length:7-9',
'emp_length:10',
'mths_since_issue_d:<38',
'mths_since_issue_d:38-39',
'mths_since_issue_d:40-41',
'mths_since_issue_d:42-48',
'mths_since_issue_d:49-52',
'mths_since_issue_d:53-64',
'mths_since_issue_d:65-84',
'mths_since_issue_d:>84',
'int_rate:<=9.548',
'int_rate:9.548-12.025',
'int_rate:12.025-15.74',
'int_rate:15.74-20.281',
'int_rate:>20.281',
'mths_since_earliest_cr_line:<140',
'mths_since_earliest_cr_line:141-184',
'mths_since_earliest_cr_line:185-264',
'mths_since_earliest_cr_line:265-352',
'mths_since_earliest_cr_line:>352',
'inq_last_6mths:0',
'inq_last_6mths:1-2',
'inq_last_6mths:3-6',
'inq_last_6mths:>6',
'acc_now_delinq:0',
'acc_now_delinq:>=1',
'total_rev_hi_lim:<=5K',
'total_rev_hi_lim:5K-10K',
'total_rev_hi_lim:10K-20K',
'total_rev_hi_lim:20K-30K',
'total_rev_hi_lim:30K-40K',
'total_rev_hi_lim:40K-55K',
'total_rev_hi_lim:55K-95K',
'total_rev_hi_lim:>95K',
'annual_inc:<20K',
'annual_inc:20K-30K',
'annual_inc:30K-40K',
'annual_inc:40K-50K',
'annual_inc:50K-60K',
'annual_inc:60K-70K',
'annual_inc:70K-80K',
'annual_inc:80K-90K',
'annual_inc:90K-100K',
'annual_inc:100K-120K',
'annual_inc:120K-140K',
'annual_inc:>140K',
'dti:<=1.4',
'dti:1.4-3.5',
'dti:3.5-7.7',
'dti:7.7-10.5',
'dti:10.5-16.1',
'dti:16.1-20.3',
'dti:20.3-22.4',
'dti:22.4-35',
'dti:>35',
'mths_since_last_delinq:Missing',
'mths_since_last_delinq:0-3',
'mths_since_last_delinq:4-30',
'mths_since_last_delinq:31-56',
'mths_since_last_delinq:>=57',
'mths_since_last_record:Missing',
'mths_since_last_record:0-2',
'mths_since_last_record:3-20',
'mths_since_last_record:21-31',
'mths_since_last_record:32-80',
'mths_since_last_record:81-86',
'mths_since_last_record:>86',
]

Reference category จาก PD model ที่ได้คัดเลือกแล้วว่ามีส่วนช่วยในการทำนายการผิดนัดชำระหนี้:

In [37]:
ref_categories_pd = ['grade:G',
'home_ownership:RENT_OTHER_NONE_ANY',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'verification_status:Verified',
'purpose:smb_edu_mov_house_renew_wedd',
'initial_list_status:f',
'term:60',
'emp_length:0',
'mths_since_issue_d:>84',     
'int_rate:>20.281',
'mths_since_earliest_cr_line:<140',
'inq_last_6mths:>6',
'acc_now_delinq:0',
'total_rev_hi_lim:<=5K',
'annual_inc:<20K',
'dti:>35',
'mths_since_last_delinq:0-3',
'mths_since_last_record:0-2']

เลือกเก็บเฉพาะตัวแปรที่ต้องใช้เท่านั้น นั่นคือตัวแปรที่มีชื่ออยู่ใน features_all list:

In [38]:
loan_data_inputs_pd_temp = loan_data_inputs_pd[features_all_pd]

ทำการ drop ตัวแปรที่เป็น reference category dummy variables:

In [39]:
loan_data_inputs_pd_temp = loan_data_inputs_pd_temp.drop(ref_categories_pd, axis = 1)

In [40]:
loan_data_inputs_pd_temp.shape

(466285, 89)

เปิด PD model (Logistic Regression model) จาก disk:

In [41]:
filehandler = open('./data/pd_model.pkl','rb')
reg_pd = pickle.load(filehandler)
filehandler.close()

ทำนาย Probability of Default (PD) ของผู้ขอกู้ใน test dataset (โดยจะไม่ทำนายเป็นผลลัพธ์ว่าผิดหรือไม่ผิดนัดชำระหนี้ แต่จะให้ผลลัพธ์คือความน่าจะเป็นดิบ หรือ raw probabilities) แล้วเลือกเก็บเฉพาะ column แรกซึ่งเป็น Probability of Default (PD):

In [42]:
reg_pd.model.predict_proba(loan_data_inputs_pd_temp)[: ][: , 0]

array([0.02946456, 0.09295859, 0.03876429, ..., 0.02431967, 0.04256716,
       0.05009425])

In [43]:
loan_data_inputs_pd['PD'] = reg_pd.model.predict_proba(loan_data_inputs_pd_temp)[: ][: , 0]

In [44]:
loan_data_inputs_pd['PD'].head()

Unnamed: 0
427211    0.029465
206088    0.092959
136020    0.038764
412305    0.210607
36159     0.196306
Name: PD, dtype: float64

แสดงสถิติของ Probability of Default (PD):

In [45]:
loan_data_inputs_pd['PD'].describe()

count    466285.000000
mean          0.109310
std           0.070863
min           0.007448
25%           0.056096
50%           0.093532
75%           0.146544
max           0.643362
Name: PD, dtype: float64

ขณะนี้:

DataFrame ชื่อ `loan_data_inputs_pd` มีผลทำนาย:
- PD: Probability of Default (ความน่าจะเป็นที่ลูกหนี้จะผิดนัดชำระหนี้)

DataFrame ชื่อ `loan_data_preprocessed` มีผลทำนาย:
- LGD: Loss Given Default (เปอร์เซนต์ความเสียหายเมื่อลูกหนี้ผิดนัดชำระหนี้)
- EAD: Exposure at Default (ยอดหนี้เมื่อลูกหนี้ผิดชำระหนี้)

ทำการรวม DataFrame ชื่อ `loan_data_inputs_pd` และ `loan_data_preprocessed`:

In [46]:
# Concatenate two DataFrames along the columns (axis = 1).
loan_data_preprocessed_new = pd.concat([loan_data_preprocessed, loan_data_inputs_pd], axis = 1)

In [47]:
loan_data_preprocessed_new.shape

(466285, 538)

In [48]:
loan_data_preprocessed_new.head()

,Unnamed: 0.1,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,...,mths_since_last_delinq:31-56,mths_since_last_delinq:>=57,mths_since_last_record:Missing,mths_since_last_record:0-2,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86,PD
0,0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,...,0,0,1,0,0,0,0,0,0,0.168813
1,1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,...,0,0,1,0,0,0,0,0,0,0.282537
2,2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,...,0,0,1,0,0,0,0,0,0,0.231528
3,3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,...,1,0,1,0,0,0,0,0,0,0.208613
4,4,4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,...,1,0,1,0,0,0,0,0,0,0.133979


## EL (Expected Loss)

$EL = PD \times LGD \times EAD$

ทำการคำนวณค่า Expected Loss (EL: ค่าความเสียหายที่คาดว่าจะเกิดขึ้น):

In [49]:
loan_data_preprocessed_new['EL'] = loan_data_preprocessed_new['PD'] * loan_data_preprocessed_new['LGD'] * loan_data_preprocessed_new['EAD']

แสดงสถิติของค่า Expected Loss (EL) ที่คำนวณได้:

In [50]:
loan_data_preprocessed_new['EL'].describe()

count    466285.000000
mean       1095.792161
std        1112.687990
min           7.881491
25%         357.645114
50%         714.194287
75%        1424.327540
max       12039.955956
Name: EL, dtype: float64

จากสถิติข้างบนพบว่า ค่าเฉลี่ยของ Expected Loss อยู่ที่ 1095.79 USD

แสดงตัวอย่างค่า PD (Probability of Default), LGD (Loss Given Default), EAD (Exposure at Default), EL (Expected Loss) ของแต่ละการกู้ยืม:

In [51]:
loan_data_preprocessed_new[['funded_amnt', 'PD', 'LGD', 'EAD', 'EL']].head()

,funded_amnt,funded_amnt,PD,LGD,EAD,EL
0,5000,5000,0.168813,0.914117,2943.860059,454.281235
1,2500,2500,0.282537,0.915901,1941.268368,502.353434
2,2400,2400,0.231528,0.919807,1577.892265,336.029567
3,10000,10000,0.208613,0.905437,6591.452793,1245.030630
4,3000,3000,0.133979,0.911931,2120.604693,259.093920


จากตารางข้างบน

ยกตัวอย่าง: index ที่ 0 มีจำนวนเงินที่กู้ยืมเท่ากับ 5000 USD และมี Expected Loss (ค่าความเสียหายที่คาดว่าจะเกิดขึ้น) เท่ากับ 454.28 USD

แสดงสถิติของค่า funded_amnt (จำนวนเงินที่ปล่อยกู้):

In [52]:
loan_data_preprocessed_new['funded_amnt'].describe()

,funded_amnt,funded_amnt
count,466285.000000,466285.000000
mean,14291.801044,14291.801044
std,8274.371300,8274.371300
min,500.000000,500.000000
25%,8000.000000,8000.000000
50%,12000.000000,12000.000000
75%,20000.000000,20000.000000
max,35000.000000,35000.000000


สถาบันการเงินจะไม่โฟกัสที่ความเสียหายที่อาจเกิดขึ้นกับผู้ขอกู้รายใดรายหนึ่ง แต่จะสนใจความเสียหายที่เป็นภาพรวมทั้งหมด ดังนั้น จึงต้องคำนวณหา Expected Loss รวมของทุกการกู้ยืม

**คำนวณค่า Total Expected Loss จากทุกการกู้ยืมใน dataset:**

In [53]:
loan_data_preprocessed_new['EL'].sum()

510951447.79853255

พบว่า ค่า Total Expected Loss เท่ากับ 510,951,447.80 USD

**คำนวณค่า Total Funded Amount จากทุกการกู้ยืมใน dataset:**

In [54]:
loan_data_preprocessed_new['funded_amnt'].sum()

funded_amnt    6664052450
funded_amnt    6664052450
dtype: int64

พบว่า ค่า Total Funded Amount เท่ากับ 6,664,052,450.00 USD

**คำนวณอัตราส่วนของ `Total Expected Loss จากทุกการกู้ยืมใน dataset` ต่อ `Total Funded Amount จากทุกการกู้ยืมใน dataset`:**

In [55]:
loan_data_preprocessed_new['EL'].sum() / loan_data_preprocessed_new['funded_amnt'].sum()

funded_amnt    0.076673
funded_amnt    0.076673
dtype: float64

นั่นคือ อัตราส่วนของ Total Expected Loss ต่อ Total Funded Amount มีค่าประมาณ 7.66%

อัตราส่วนนี้ควรต่ำกว่าทุนสำรองของสถาบันการเงิน เพื่อให้สามารถ abdorb ความเสียหายจากการผิดนัดชำระหนี้ อัตราส่วนนี้จะเปลี่ยนแปลงไปตาม risk management ที่สถาบันการเงินตัดสินใจว่าจะใช้นโยบายที่ conservative หรือ aggressive ในการปล่อยกู้ในอนาคต

---

# End of notebook.

---